# <center> 2차 크롤링 - <원티드> </center>

##### 링크들 들어가서 각각 정보 수집, 딥러닝

In [1]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib


In [2]:
options = webdriver.ChromeOptions()

url = "https://www.wanted.co.kr/jobsfeed"
driver = webdriver.Chrome(service=Service("../../driver/chromedriver"),
                          options=options)

driver.get(url)

In [18]:
driver.get(url)

In [19]:
nav_element = driver.find_element(By.CLASS_NAME, "Aside_asideList__3WNyu")
search_element = nav_element.find_element(By.CLASS_NAME, "SvgIcon_SvgIcon__root__8vwon")

search_element.click()

In [20]:
search_doc = driver.find_element(By.CLASS_NAME, "SearchInput_SearchInput__gySrv")
search_doc.send_keys('딥러닝')
search_doc.send_keys(Keys.ENTER)


In [21]:
wait = WebDriverWait(driver, 10)
position_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(), "포지션")]')))
position_tab.click()

In [22]:
simple_list = []
wait = WebDriverWait(driver, 10)  

simple_name_elements = []  

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    new_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')
    simple_name_elements.extend(new_elements)

    # StaleElementReferenceException을 처리하기 위해 엘리먼트 수의 변경을 확인
    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    if len(simple_name_elements) >= 20:
        break


simple_names = [element.text.strip() for element in simple_name_elements]
print(simple_names)
print(len(simple_names))

['', '', '', '', '', '', '', '', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국', '경기 · 한국', '경기 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국']
41


In [23]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
# 기업이름 요소 초기 로드
company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
num_elements_before_scroll = len(company_name_elements)
while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break
    # 현재 로드된 회사명 요소들 가져오기
    company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
    num_elements_before_scroll = len(company_name_elements)
    if num_elements_before_scroll >= 20:
        break
company_names = []
href_lists = []
href_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-attribute-id="position__click"]')
# 가져온 데이터 중에서 조건에 맞는 것들만 필터링
for i, element in enumerate(company_name_elements):
    simple_name = simple_names[min(i, len(simple_names) - 1)]
    company_name_text = element.text.strip()
    # simple_name에 '서울 · 한국' 또는 '경기 · 한국'이 포함된 경우에만 추가
    if '서울 · 한국' in simple_name or '경기 · 한국' in simple_name:
        company_names.append(company_name_text)
        href_value = href_elements[i].get_attribute('href')
        href_lists.append(href_value)



In [24]:
#직무(담당업무 열)가져오기
job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')
num_elements_before_scroll = len(company_name_elements)
while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break
    job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')
    num_elements_before_scroll = len(company_name_elements)
    if num_elements_before_scroll >= 20:
        break
job_titles = []
for i, element in enumerate(job_title_elements):
    # simple_names 리스트에서 현재 인덱스에 해당하는 값 가져오기
    simple_name = simple_names[min(i, len(simple_names)-1)]
    # 만약 값이 '서울'이나 '경기'로 시작하면 추가
    if simple_name.startswith('서울') or simple_name.startswith('경기'):
        job_titles.append(job_title_elements[i].text.strip())  # 수정된 부분

In [8]:

# 기업주소 리스트 초기화
address_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기업 주소를 포함한 <section> 요소 찾기
        address_div = soup.find('section', class_='JobWorkPlace_className__ra6rp')

        if address_div:
            # <section> 내부의 주소 정보를 찾아서 리스트에 추가
            address_items = address_div.find_all('span', class_='body')
            address = [address_item.text.strip() for address_item in address_items]

            # URL과 해당하는 주소 리스트를 한 리스트에 추가
            address_list.append(address)
            break  # 주소를 찾았으므로 루프 종료

        # 스크롤을 크게 아래로 내림 (300픽셀씩)
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)  # 스크롤 후 충분한 시간 대기



In [9]:
# address_list에 있는 각 기업주소에 대해 처리
for i in range(len(address_list)):
    # 첫 번째 요소를 제외한 나머지 주소로 갱신
    address_list[i] = address_list[i][1:]

# 리스트 출력
for address in address_list:
    print(f'{address}\n')

print(len(address_list))

['서초구 강남대로 527, 브랜드칸 타워 6층']

['서울특별시 강남구 역삼1동 823-30 라인빌딩 10층, 11층, 12층']

['서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층']

['서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층']

['성남시 수정구 금토로80번길 37(금토동)']

['구로구 디지털로 242 \u200b(구로한화비즈메트로1차) 709호']

['서울시 성동구 성수일로 77 서울숲 IT 밸리 805호']

['강남구 테헤란로 217, 오렌지플래닛']

['서울특별시 중구 소월로2길 30, T타워 18층']

['서울 강남구 테헤란로 43길 18, SC빌딩 6층']

['서울 마포구 마포대로 144 (공덕동, 마포T타운) 8층']

['서울시 강남구 삼성로 85길 26, V&S 601호']

['강남구 언주로 544, 4층 테서']

['경기도 김포시 양촌읍 황금로 323번길 26번지']

['강남구 언주로 527 강남텔레피아빌딩 4층']

['강남구 언주로 527 강남텔레피아빌딩 4층']

['경기 성남시 분당구 벌말로 50번길 41 투아이센터 7층']

['경기도 성남시 분당구 분당내곡로 131, 7층(백현동, 판교테크원)']

['경기도 성남시 수정구 금토로 80번길 11 (금토동) 판교이노베이션랩 지식산업센터 1409 /1410/ 1411호']

['경기도 성남시 수정구 금토로 80번길 11 (금토동) 판교이노베이션랩 지식산업센터 1409 /1410/ 1411호']

['경기도 성남시 수정구 금토로 80번길 11 (금토동) 판교이노베이션랩 지식산업센터 1409 /1410/ 1411호']

['서초구 서초대로38길 12 마제스타시티 타워1 10층 (주)보이저엑스']

['서울시 성동구 연무장5가길 7 현대테라스타워 East동 13층']

['서울 성동구 성수이로22길 37 (성수동2가) 아크밸리 지식산업 센터 303c호 안티그래비티']

['서울특별시 강남구 삼성로 571 K빌딩 8층']

['용인시 기흥구'

In [10]:
#연봉
salary_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)


    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기술 스택을 포함한 <div> 요소 찾기
        salary_div = soup.find('div', class_='Tags_tagsClass__mvehZ')

        if salary_div:
            salary_items = salary_div.find_all('li')
            first_li = salary_items[0] if salary_items else None

            if first_li and "연봉" in first_li.text:
                salaries = [salary_item.text.strip().replace("#", "") for salary_item in salary_items]
                salary_list.append(salaries[0])
                break
            else:
                salary_list.append(["-"])
                break


     
# 리스트 출력
for i in salary_list:
    print(f'{i}\n')

print(len(salary_list))

['-']

연봉업계평균이상

['-']

['-']

연봉업계평균이상

['-']

연봉업계평균이상

['-']

['-']

['-']

['-']

['-']

['-']

['-']

['-']

['-']

['-']

연봉업계평균이상

연봉업계평균이상

연봉업계평균이상

연봉업계평균이상

연봉업계평균이상

['-']

['-']

연봉업계평균이상

['-']

연봉업계평균이상

['-']

['-']

['-']

['-']

['-']

['-']

33


In [15]:
# 자격요건 텍스트를 저장할 리스트 초기화
qualifications_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "자격요건" 부분을 추출
        qualifications_header = soup.find('h3', string='자격요건')

        if qualifications_header:
            # "자격요건" 부분이 포함된 <span> 요소 찾기
            qualifications_span = qualifications_header.find_next('p').find('span')

            if qualifications_span:
                # <span> 요소에서 텍스트 추출
                qualifications_text = qualifications_span.get_text(strip=True)
                # 리스트에 자격요건 추가
                qualifications_list.append(qualifications_text)
                break  # 자격요건을 찾았으므로 루프 종료
            else:
                print('자격요건이 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for qualifications in qualifications_list:
    print(f'{qualifications}\n')

print(len(qualifications_list))

■ 이런 분과 함께하고 싶습니다- 풍부한 SI 영업, AI 솔루션 영업 경험이 있으신 분- 적극적으로 사업 기회를 발굴하고 가치 제안을 하실 수 있는 분

+ 인공지능 및 머신러닝 연구 개발 경험+ Python, Pytorch, Tensorflow 사용 가능

- OCR 시스템의 구조를 알고 이를 딥러닝 모형으로 구현이 가능하신 분- 딥러닝 서비스 배포를 위한 기본적인 구조를 알고 계신 분- 딥러닝 프레임워크 (Pytorch, Tensorflow 등) 사용에 익숙하신 분- 딥러닝 방식을 적용한 OCR 문자 인식 경험이 있으신 분- 기존 OCR 모델을 최적화 혹은 고도화 시킨 경험이 있으신 분

• 학력 : 석사 이상• 딥러닝, 머신러닝에 대한 깊은 이해를 기반으로 한 AI 서비스 개발 경험자• 3D 데이터셋 분석 및 처리 역량 보유 (Open3d 등)• Pytorch 기반 해당 분야 최신 딥러닝 논문 기술에 대한 (ex. Latent Diffusion, NeRF, Instant-NGP, Dreamfusion, LION 등) 구현 및 응용 역량 보유

• 관련 분야 3년 이상 경력자• C/C++/Python 숙련자• Tensorflow/Pytorch 숙련자• 머신러닝/딥러닝 이론에 대한 숙련된 이해도

■ 자격요건- Computer vision 또는 Machine learning 관련 석사학위 이상 보유 또는 관련 경력 2년 이상의 학사학위 보유자- Computer vision 관련 프로젝트 수행 경험자- Python 개발 가능자

• 나이/성별 : 무관• 군필(남성의 경우)• 신입 : AI 관련 학과 대학원 졸 이상• 경력 : 프로그래밍 관련 학과 대학교 졸업 및 AI 경력 1년 이상(대학원 AI 전공자의 경우 AI 경력 2년 인정)

• AI, CS 관련 학사 이상의 학위를 보유하신 분• 딥러닝 모델 개발 프로젝트 경험이 있으신 분• Python, Torch 사용에 능숙하신 분

딥러닝 경력 2년 이상 (석사 이상 경력 인정)PyTorch, Tensorflo

In [29]:
#webdriver오류용
driver = webdriver.Chrome()

In [30]:
#담당업무
work_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "주요업무" 부분을 추출
        work_header = soup.find('h3', string='주요업무')

        if work_header:
            work_span = work_header.find_next('p').find('span')

            if work_span:
                # <span> 요소에서 텍스트 추출
                work_text = work_span.get_text(strip=True)
                work_list.append(work_text)
                break  
            else:
                print('주요업무가 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for work in work_list:
    print(f'{work}\n')

print(len(work_list))

해당 포지션은 세일즈/전략 파트의 파트장으로서 AI 솔루션 및 3D DX 등 한국딥러닝의 세일즈와 사업전략을 총괄하는 자리입니다.단기적으로는 고객 발굴부터 계약체결까지의 B2B 세일즈 전반을 총괄합니다.중장기적으로는 세일즈/전략 파트의 팀을 구성하고 리딩하며, 시장 상황에 맞춰 영업전략 및 사업전략을 구상하고 실행시킵니다.저희 한국딥러닝의 솔루션들이 기업들의 어떤 Pain Point를 해결해 줄 수 있을지 분석하여 고객과 회사가 모두 Win-Win할 수 있는 전략을 이끌어내는 핵심적인 포지션입니다.■ 이런 일을 하게 됩니다- 전사 세일즈 전략 및 프로세스를 확립하고 미래 방향성을 설정합니다.- AI 솔루션과 3D DX 기술을 기업에서 어떤 방식으로 활용할 수 있을지 분석합니다.- 다양한 분야의 기업들을 대상으로 사업/영업 방법을 구상합니다. (민간 B2B, 파트너십, 입찰, R&D 사업, 정부지원 사업 등)- 잠재고객을 대상으로 제안부터 계약 체결까지 AtoZ를 리드합니다.- 세일즈/전략팀을 구성하고 코칭 및 관리를 통해 성장과 성과를 이끌어냅니다.

- 최신의 딥러닝 학습 모델 개발, 최적화- 대규모 언어모델 학습 및 최적화- 대규모 언어모델 학습 데이터 수집 및 전처리

- Text Detection / Text Recognition / E2E OCR 모형 개발- Document Understanding- 모형 최적화- Edge 장비에 맞는 경량화 기술 연구- Table Parsing- Watermark 제거

• 딥러닝 기반 3D Reconstruction, Generation 분야 리서치• 3D-Text Multimodal (Clip-Forge 등) 분야 응용 및 리서치• 데이터 구축 및 딥러닝 아키텍처 설계 및 구축

• 머신러닝/딥러닝 관련 개발• 머신러닝/딥러닝 서비스 운영 시스템 설계 및 구현

■ 담당업무 :- Computer vision 분야의 선행 기술 조사 및 연구- Computer vision 관련 알고리즘 개발- 특허 등 기술문서 작성

In [31]:
#우대사항
preferential_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    try:
        # 기다리기를 설정하여 페이지 로딩을 기다림
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    except TimeoutException:
        print('페이지 로딩이 제때 완료되지 않았습니다.')

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "우대사항" 부분을 추출
        preferential_header = soup.find('h3', string='우대사항')

        if preferential_header:
            preferential_span = preferential_header.find_next('p').find('span')

            if preferential_span:
                # <span> 요소에서 텍스트 추출
                preferential_text = preferential_span.get_text(strip=True)
                preferential_list.append(preferential_text)
                break  
            else:
                break 
        else:
            # 우대사항이 페이지에 없으면 리스트에 공백 추가
            preferential_list.append('')
            break

for preferential in preferential_list:
    print(f'{preferential}\n')

print(len(preferential_list))

■ 이런 분이면 더 좋습니다.- 다양한 산업 분야에 걸쳐 네트워크를 갖고 계신 분- 컴퓨터 공학 지식 및 AI 기술에 대한 높은 수준의 이해가 있으신 분

+ 컴퓨터 공학 학사 또는 이상의 학위+ 딥러닝 관련 연구실 출신 우대



• 3D Generation, 3D Reconstruction 방법론에 대한 새로운 아이디어가 있으신 분• Git-flow 및 프로젝트 관리 툴을 통한 플랜 설정 및 협업 능력을 보유하신 분• Docker, Kubernetes를 통한 학습, 추론 서버 구축 및 관리 역량을 보유하신 분

• 딥러닝 프레임워크 저수준 활용 및 디버깅 경험• NLP 모델 개발 및 튜닝 경험• 딥러닝 모델 경량화 경험• 딥러닝 적대적 샘플 생성 등 모델 검증 방법론에 대한 연구 및 고민 경험



• 컴퓨터/시스템 공학, 전기/전자공학, 수학/통계학 전공자• 통계 분석 능숙자, 컴퓨터 활용 능력 우수

• Generative AI 관련 개발 경험이 있으신 분• AI 관련 석사 학위를 보유하신 분• 유관 경력 3년 이상이신 분• 멀티모달 AI 개발에 관심있으신 분• GCP, Git, Docker 개발 환경에 능숙하신 분

딥러닝 혹은 유사 분야 논문 작성 경험논문 해석 및 Oral Presentation 이해 가능한 수준의 영어 실력새로운 연구에 대해 적극적이고 주도적인 분Product/Data Centric AI를 경험해보신 분최신 기술 트렌드를 빨리 캐치하고 적용할 수 있는 분

- 상용 딥러닝 모델 구현 경험이 있으신 분- MLflow, WandB, Neptune 관련 기술 및 경험이 있으신 분- Docker, Kubernetes 환경에서 연구 및 개발 경험이 있으신 분- MLOps 에 대한 관심 혹은 경험- 데이터의 통계적 분석과 모델 분석을 통한 insight가 있으신 분- 스스로 문제를 파악하고 가설을 제기하고 실험을 통해 검증하는 것을 좋아하는 분- 패션 이미지 데이터에 대한 관심이 많은 분현재 사용하는 기술- DL: Pytorch- VCS: 

In [42]:
driver.quit()

In [32]:
# 같은 인덱스끼리 합치기(자격요건 + 우대사항 + 담당업무)
required_list = [qual + pref + work for qual, pref, work in zip(qualifications_list, preferential_list, work_list)]
for item in required_list:
    print(item)

■ 이런 분과 함께하고 싶습니다- 풍부한 SI 영업, AI 솔루션 영업 경험이 있으신 분- 적극적으로 사업 기회를 발굴하고 가치 제안을 하실 수 있는 분■ 이런 분이면 더 좋습니다.- 다양한 산업 분야에 걸쳐 네트워크를 갖고 계신 분- 컴퓨터 공학 지식 및 AI 기술에 대한 높은 수준의 이해가 있으신 분해당 포지션은 세일즈/전략 파트의 파트장으로서 AI 솔루션 및 3D DX 등 한국딥러닝의 세일즈와 사업전략을 총괄하는 자리입니다.단기적으로는 고객 발굴부터 계약체결까지의 B2B 세일즈 전반을 총괄합니다.중장기적으로는 세일즈/전략 파트의 팀을 구성하고 리딩하며, 시장 상황에 맞춰 영업전략 및 사업전략을 구상하고 실행시킵니다.저희 한국딥러닝의 솔루션들이 기업들의 어떤 Pain Point를 해결해 줄 수 있을지 분석하여 고객과 회사가 모두 Win-Win할 수 있는 전략을 이끌어내는 핵심적인 포지션입니다.■ 이런 일을 하게 됩니다- 전사 세일즈 전략 및 프로세스를 확립하고 미래 방향성을 설정합니다.- AI 솔루션과 3D DX 기술을 기업에서 어떤 방식으로 활용할 수 있을지 분석합니다.- 다양한 분야의 기업들을 대상으로 사업/영업 방법을 구상합니다. (민간 B2B, 파트너십, 입찰, R&D 사업, 정부지원 사업 등)- 잠재고객을 대상으로 제안부터 계약 체결까지 AtoZ를 리드합니다.- 세일즈/전략팀을 구성하고 코칭 및 관리를 통해 성장과 성과를 이끌어냅니다.
+ 인공지능 및 머신러닝 연구 개발 경험+ Python, Pytorch, Tensorflow 사용 가능+ 컴퓨터 공학 학사 또는 이상의 학위+ 딥러닝 관련 연구실 출신 우대- 최신의 딥러닝 학습 모델 개발, 최적화- 대규모 언어모델 학습 및 최적화- 대규모 언어모델 학습 데이터 수집 및 전처리
- OCR 시스템의 구조를 알고 이를 딥러닝 모형으로 구현이 가능하신 분- 딥러닝 서비스 배포를 위한 기본적인 구조를 알고 계신 분- 딥러닝 프레임워크 (Pytorch, Tensorflow 등) 사용에 익숙하신 분- 딥러닝 방

In [33]:
keyword_list = ["딥러닝"] * len(company_names)

print(len(keyword_list))
print(keyword_list)

33
['딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝', '딥러닝']


In [34]:
import pandas as pd
# 데이터프레임 생성
data = {
    '기업이름': company_names,
    '연봉': salary_list,
    '기업주소': address_list,
    '키워드': keyword_list,
    '담당업무': job_titles,
    '필요역량': required_list
}
df = pd.DataFrame(data)
# 데이터프레임 출력
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,한국딥러닝,[-],"[서초구 강남대로 527, 브랜드칸 타워 6층]",딥러닝,세일즈/전략 파트장,"■ 이런 분과 함께하고 싶습니다- 풍부한 SI 영업, AI 솔루션 영업 경험이 있으..."
1,포지큐브(POSICUBE),연봉업계평균이상,"[서울특별시 강남구 역삼1동 823-30 라인빌딩 10층, 11층, 12층]",딥러닝,언어모델 딥러닝 개발자,"+ 인공지능 및 머신러닝 연구 개발 경험+ Python, Pytorch, Tenso..."
2,한국딥러닝,[-],"[서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층]",딥러닝,ML/DL Engineer (머신러닝/딥러닝 엔지니어),- OCR 시스템의 구조를 알고 이를 딥러닝 모형으로 구현이 가능하신 분- 딥러닝 ...
3,한국딥러닝,[-],"[서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층]",딥러닝,딥러닝 엔지니어 (3D Reconstruction / 3D Generation),"• 학력 : 석사 이상• 딥러닝, 머신러닝에 대한 깊은 이해를 기반으로 한 AI 서..."
4,슈어소프트테크,연봉업계평균이상,[성남시 수정구 금토로80번길 37(금토동)],딥러닝,[AI기술팀] 머신러닝/딥러닝 개발자,• 관련 분야 3년 이상 경력자• C/C++/Python 숙련자• Tensorflo...


In [35]:
# • 문자를 ,으로 바꾸기
df['담당업무'] = df['담당업무'].str.replace('•', ',')
df['필요역량'] = df['필요역량'].str.replace('•', ',')

# 작은 따옴표 및 괄호 제거
df['연봉'] = df['연봉'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df['기업주소'] = df['기업주소'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,한국딥러닝,-,"서초구 강남대로 527, 브랜드칸 타워 6층",딥러닝,세일즈/전략 파트장,"■ 이런 분과 함께하고 싶습니다- 풍부한 SI 영업, AI 솔루션 영업 경험이 있으..."
1,포지큐브(POSICUBE),연봉업계평균이상,"서울특별시 강남구 역삼1동 823-30 라인빌딩 10층, 11층, 12층",딥러닝,언어모델 딥러닝 개발자,"+ 인공지능 및 머신러닝 연구 개발 경험+ Python, Pytorch, Tenso..."
2,한국딥러닝,-,"서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층",딥러닝,ML/DL Engineer (머신러닝/딥러닝 엔지니어),- OCR 시스템의 구조를 알고 이를 딥러닝 모형으로 구현이 가능하신 분- 딥러닝 ...
3,한국딥러닝,-,"서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층",딥러닝,딥러닝 엔지니어 (3D Reconstruction / 3D Generation),", 학력 : 석사 이상, 딥러닝, 머신러닝에 대한 깊은 이해를 기반으로 한 AI 서..."
4,슈어소프트테크,연봉업계평균이상,성남시 수정구 금토로80번길 37(금토동),딥러닝,[AI기술팀] 머신러닝/딥러닝 개발자,", 관련 분야 3년 이상 경력자, C/C++/Python 숙련자, Tensorflo..."


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업이름    33 non-null     object
 1   연봉      33 non-null     object
 2   기업주소    33 non-null     object
 3   키워드     33 non-null     object
 4   담당업무    33 non-null     object
 5   필요역량    33 non-null     object
dtypes: object(6)
memory usage: 1.7+ KB


In [37]:
import re

# 경력 정보 추출을 위한 정규표현식
career_pattern = re.compile(r'(\d+년.*?)')

# DataFrame의 각 행에 대해 경력 정보 추출 및 새로운 열에 추가
df['고용형태'] = df['필요역량'].apply(lambda x: career_pattern.findall(x)[0] if career_pattern.findall(x) else '-')

# 결과 출력
print(df[['고용형태']])

   고용형태
0     -
1     -
2     -
3     -
4    3년
5    2년
6    1년
7    3년
8    2년
9    2년
10   3년
11    -
12    -
13   1년
14    -
15   3년
16   4년
17    -
18  10년
19    -
20   3년
21    -
22    -
23   5년
24   3년
25    -
26    -
27   1년
28    -
29    -
30   2년
31    -
32  24년


In [38]:
df = df[['기업이름','고용형태','연봉','기업주소','키워드','담당업무','필요역량' ]]

In [39]:
df.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,한국딥러닝,-,-,"서초구 강남대로 527, 브랜드칸 타워 6층",딥러닝,세일즈/전략 파트장,"■ 이런 분과 함께하고 싶습니다- 풍부한 SI 영업, AI 솔루션 영업 경험이 있으..."
1,포지큐브(POSICUBE),-,연봉업계평균이상,"서울특별시 강남구 역삼1동 823-30 라인빌딩 10층, 11층, 12층",딥러닝,언어모델 딥러닝 개발자,"+ 인공지능 및 머신러닝 연구 개발 경험+ Python, Pytorch, Tenso..."
2,한국딥러닝,-,-,"서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층",딥러닝,ML/DL Engineer (머신러닝/딥러닝 엔지니어),- OCR 시스템의 구조를 알고 이를 딥러닝 모형으로 구현이 가능하신 분- 딥러닝 ...
3,한국딥러닝,-,-,"서울특별시 서초구 강남대로 537, 브랜드칸 타워 6층",딥러닝,딥러닝 엔지니어 (3D Reconstruction / 3D Generation),", 학력 : 석사 이상, 딥러닝, 머신러닝에 대한 깊은 이해를 기반으로 한 AI 서..."
4,슈어소프트테크,3년,연봉업계평균이상,성남시 수정구 금토로80번길 37(금토동),딥러닝,[AI기술팀] 머신러닝/딥러닝 개발자,", 관련 분야 3년 이상 경력자, C/C++/Python 숙련자, Tensorflo..."


In [41]:
import csv

df.to_csv('output3.csv', index=False)